In [ ]:
from oemof import solph

from ensys.components import EnInvestment

bus1 = solph.Bus(label="Bus1")
bus2 = solph.Bus(label="Bus2")

es = solph.EnergySystem()
es.add(bus1, bus2)

example_invest = EnInvestment(
    maximum=10000,
    minimum=100,
    ep_costs=1239.3,
    existing=85,
    nonconvex=False,
    offset=0
)

oe_invest = example_invest.to_oemof(es)
print(f"example_invest: {example_invest.ep_costs}")
print(f"oe_invest: {oe_invest.ep_costs}")

In [ ]:
from oemof.solph import sequence

ep_costs = 1239.3
seq_ep_costs = sequence(ep_costs)

print(f"ep_costs: {ep_costs}")
print(f"seq_ep_costs: {seq_ep_costs}")
print(f"{type(seq_ep_costs)}")

In [40]:
from backend.app.types import OepTypes
from oemof.tools import economics
import os
import pandas as pd
import json

block_schema = "onshore_wind_power_plant_north_th"
flow_ep_costs: float = 90234.2394
simulation_year: int = 2025

oep_type = OepTypes[block_schema]

root_path = os.path.abspath(
    os.path.join(os.getcwd(), "..", "backend", "data", "oep", oep_type.value[1].lower())
)

port_data_path = os.path.join(root_path, "ports", f"{block_schema}.csv")
parameter_data_path = os.path.join(root_path, "parameter", f"{block_schema}.csv")
timeseries_data_path = os.path.join(root_path, "timeseries", f"{block_schema}.csv")
#print(f"port_data_path: {port_data_path} \n parameter_data_path: {parameter_data_path} \n timeseries_data_path: {timeseries_data_path}")

# Einlesen der Portdaten
if not os.path.isfile(port_data_path):
    raise Exception("File not found.")
else:
    with open(port_data_path, "r") as f:
        ports_data = pd.read_csv(f, index_col=0, decimal=",", delimiter=";").to_dict(orient="records")

# Einlesen der Parameter
if not os.path.isfile(parameter_data_path):
    raise Exception("File not found.")
else:
    with open(parameter_data_path, "r") as f:
        parameter_data = pd.read_csv(f, index_col=0, decimal=",", delimiter=";").to_dict(orient="index")

# Einlesen der Zeitreihe
if not os.path.isfile(timeseries_data_path) and (
        oep_type.value[1].lower() == "sink" or oep_type.value[1].lower() == "source"):
    raise Exception("File not found.")
elif not os.path.isfile(timeseries_data_path):
    timeseries_data = None
else:
    with open(timeseries_data_path, "r") as f:
        timeseries_data = list(pd.read_csv(f, index_col=0, decimal=",", delimiter=";").loc[:, "data"])

parameter_year_select: dict = parameter_data[simulation_year]

# hier hat man die Parameter für das ausgewählte Jahr eingelesen
param_keys = parameter_year_select.keys()

# ep_costs berechnen für den Flow
if "investment_costs" in param_keys and \
        "interest_rate" in param_keys and \
        "operating_costs" in param_keys and \
        "lifetime" in param_keys:
    # Calculate EPC costs for flow
    capex = parameter_year_select["investment_costs"]
    opex = parameter_year_select["investment_costs"] * (parameter_year_select["operating_costs"] / 100)
    interest_rate = parameter_year_select["interest_rate"] / 100

    annuity = economics.annuity(
        capex=capex,
        wacc=interest_rate,
        n=parameter_year_select["lifetime"]
    )
    flow_ep_costs = annuity + opex

# Anlegen der Flow-Daten mit Zeitreihe oder ohne
if timeseries_data is not None:
    flow_data = {
        "investment": {
            "ep_costs": flow_ep_costs,
        },
        "fix": timeseries_data
    }
else:
    flow_data = {
        "investment": {
            "ep_costs": flow_ep_costs
        }
    }

# flow-daten die wichtig sind, zum Filtern bei der storage-daten
flow_data_keys = ["nominal_value",
                  "variable_costs",
                  "min",
                  "max",
                  "fix"
                  "positive_gradient_limit",
                  "negative_gradient_limit",
                  "full_load_time_max",
                  "full_load_time_min",
                  "integer",
                  "nonconvex",
                  "fixed_costs",
                  "age",
                  "lifetime"]

# Löschen der nicht relevanten Einträge
parameter_ys_cleaned = parameter_year_select
for key in ["investment_costs", "operating_costs", "lifetime", "interest_rate", "efficiency_el",
            "efficiency_th"]:
    if key in parameter_ys_cleaned.keys():
        del parameter_ys_cleaned[key]

for key in flow_data_keys:
    if key in parameter_ys_cleaned.keys() and not key in flow_data.keys():
        flow_data[key] = parameter_ys_cleaned[key]
        del parameter_ys_cleaned[key]
    elif key in parameter_ys_cleaned.keys() and key in flow_data.keys():
        del parameter_ys_cleaned[key]

# TODO: how do i write the flow data in the right port? Sink/Source/Rest
for port in ports_data:
    temp_investment = port["investment"]
    del port["investment"]

    if (oep_type.value[1].lower() == "sink" and port["type"] == "input") or (
            oep_type.value[1].lower() == "source" and port["type"] == "output"):
        # Sink / Source
        port["flow_data"] = flow_data
    elif port["type"] == "output" and port["name"] == "electricity":
        port["flow_data"] = flow_data
    else:
        print(f"Port: {port}")

    if "efficiencey_el" in param_keys and port["name"] == "electricity" and port["type"] == "output":
        port["efficiency"] = parameter_year_select["efficiency_el"]

    if "efficiencey_th" in param_keys and port["name"] == "heat" and port["type"] == "output":
        port["efficiency"] = parameter_year_select["efficiency_th"]

sorted_port_data = {
    "inputs": [],
    "outputs": []
}

for item in ports_data:
    tmp_type = item["type"]
    del item["type"]

    if tmp_type == "input":
        sorted_port_data["inputs"].append(item)
    elif tmp_type == "output":
        sorted_port_data["outputs"].append(item)

return_data = [{
    "node_data": parameter_ys_cleaned,
    "ports_data": sorted_port_data
}]

print(f"{json.dumps(return_data[0], indent=4)}")

{
    "node_data": {},
    "ports_data": {
        "inputs": [],
        "outputs": [
            {
                "name": "electricity",
                "flow_data": {
                    "investment": {
                        "ep_costs": 13.02425871906913
                    },
                    "fix": [
                        0.03417525,
                        0.03337725,
                        0.012131,
                        0.02673775,
                        0.038689,
                        0.038689,
                        0.02585225,
                        0.04939925,
                        0.050607,
                        0.2231075,
                        0.1995375,
                        0.15705,
                        0.206335,
                        0.25902,
                        0.2267275,
                        0.410055,
                        0.2814225,
                        0.25902,
                        0.175745,
                        0.15705